# Backtesting Using BackTrader

In [1]:
import backtrader as bt
import datetime
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
class PandasData(bt.feeds.PandasData):
    lines = ('open', 'close')
    params = (
        ('datetime', None),  # use index as datetime
        ('open', 0),         # the [0] column is open price
        ('close', 1),        # the [1] column is close price
        ('high', 0),
        ('low', 0),
        ('volume', 0),
        ('openinterest', 0),
    )

In [9]:
class BLStrategy(bt.Strategy):
    # list for tickers
    params = (
        ('stocks', []),
    )


    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    
    def __init__(self, weights):
        self.datafeeds = {}
        self.weights = weights                              # weights for all stocks
        self.committed_cash = 0
        self.bar_executed = 0

        # price data and order tracking for each stock
        for i, ticker in enumerate(self.params.stocks):
            self.datafeeds[ticker] = self.datas[i]


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            print(f"Order for {order.size} shares of {order.data._name} at {order.created.price} is {order.getstatusname()}")

        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"Bought {order.executed.size} shares of {order.data._name} at {order.executed.price}, cost: {order.executed.value}, comm: {order.executed.comm}")
            elif order.issell():
                print(f"Sold {order.executed.size} shares of {order.data._name} at {order.executed.price}, cost: {order.executed.value}, comm: {order.executed.comm}")


        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f'Order for {order.size} shares of {order.data._name} at {order.created.price} is {order.getstatusname()}')


    # for each date, place orders according to the weights, this part has some bugs
    def next(self):
        date = self.data.datetime.date(0)
        weights = self.weights.loc[date.strftime('%Y-%m-%d')]

        if not self.position:
            print("We do not hold any positions at the moment")
        self.log(f"Total portfolio value: {self.broker.getvalue()}")

        for ticker in self.params.stocks:
            # Calculate the target value for this stock based on the target percentage
            data = self.datafeeds[ticker]
            target_percent = weights[ticker]
                
            self.log(f"{ticker} Open: {data.open[0]}, Close: {data.close[0]}, Target Percent: {target_percent}")
            self.orders = self.order_target_percent(data, target=target_percent)
        
        
if __name__ == '__main__':
    # load price and weights data
    close_prices_df = pd.read_csv('../data/synthetic_close_prices.csv', index_col='Date', parse_dates=True)
    open_prices_df = pd.read_csv('../data/synthetic_open_prices.csv', index_col='Date', parse_dates=True)
    weights_df = pd.read_csv('../data/synthetic_weights.csv', index_col='Date', parse_dates=True)
    weights_df = weights_df / weights_df.sum(axis=1).values.reshape(-1, 1) * 0.9

    # Combine open and close prices into one DataFrame
    combined_df = open_prices_df.join(close_prices_df, lsuffix='_open', rsuffix='_close')
    combined_df = combined_df.dropna()
    # combined_df = combined_df.head()
    # display(combined_df.head())
    
    # align the date of price and weights
    weights_df = weights_df.loc[combined_df.index]

    # initialize cerebro engine
    cerebro = bt.Cerebro()

    # read data feeds
    for col in close_prices_df.columns:
        data = PandasData(dataname=combined_df[[col + '_open', col + '_close']])
        cerebro.adddata(data, name=col)

    # strategy setting
    cerebro.broker.setcash(100000)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.broker.set_shortcash(True)
    cerebro.addstrategy(BLStrategy, weights=weights_df, stocks=close_prices_df.columns)

    # analyze strategy
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='AnnualReturn')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.03, annualize=True, _name='SharpeRatio')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')
    
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()
    print('Final Portfolio Value:', cerebro.broker.getvalue())
    cerebro.plot()



Starting Portfolio Value: 100000.00
We do not hold any positions at the moment
2024-01-01, Total portfolio value: 100000.0
2024-01-01, Stock1 Open: 107.72579552117703, Close: 100.0, Target Percent: -0.18000000000000002
2024-01-01, Stock2 Open: 105.9556751660438, Close: 100.0, Target Percent: 0.18000000000000002
2024-01-01, Stock3 Open: 96.89456788248314, Close: 100.0, Target Percent: 0.36000000000000004
2024-01-01, Stock4 Open: 105.03725157153512, Close: 100.0, Target Percent: 0.36000000000000004
2024-01-01, Stock5 Open: 101.13349884713404, Close: 100.0, Target Percent: 0.18000000000000002
Order for -180 shares of Stock1 at 100.0 is Submitted
Order for 180 shares of Stock2 at 100.0 is Submitted
Order for 360 shares of Stock3 at 100.0 is Submitted
Order for 360 shares of Stock4 at 100.0 is Submitted
Order for 180 shares of Stock5 at 100.0 is Submitted
Order for -180 shares of Stock1 at 100.0 is Accepted
Order for 180 shares of Stock2 at 100.0 is Accepted
Order for 360 shares of Stock3 a

<IPython.core.display.Javascript object>

In [ ]:
# date range
start_date = datetime.date(2024, 1, 1)
end_date = start_date + datetime.timedelta(days=99)
date_range = pd.bdate_range(start=start_date, end=end_date)

# fake price for 5 stocks
num_days = len(date_range)
num_stocks = 5
prices = pd.DataFrame(np.random.normal(loc=100, scale=10, size=(num_days, num_stocks)), index=date_range, columns=[f'Stock{i}' for i in range(1, num_stocks+1)])
prices = prices.reset_index().rename(columns={"index": "Date"})

# fake weights for 5 stocks
weights = np.random.uniform(-1, 1, (num_days, num_stocks))
weights = weights / weights.sum(axis=1, keepdims=True)
weights = pd.DataFrame(weights, index=date_range, columns=[f'Stock{i}' for i in range(1, num_stocks+1)])
weights = weights.reset_index().rename(columns={"index": "Date"})


# save price and weights
# prices.to_csv('../data/synthetic_open_prices.csv')
# weights.to_csv('../data/synthetic_weights.csv')

print("Data generated and saved successfully.")

print(prices.head())
print(weights.head())

Data generated and saved successfully.
        Date      Stock1      Stock2      Stock3      Stock4      Stock5
0 2024-01-01   89.278044   72.770513   92.084425  107.102840   87.580582
1 2024-01-02  107.708380  110.863840  119.725076   94.094580   88.800516
2 2024-01-03   84.115801   99.211717   99.890973   88.683592   96.493743
3 2024-01-04  104.471406   93.874793   91.871149  104.149267  101.779381
4 2024-01-05   83.653062   91.563991  111.332034  105.648394  103.329560
        Date    Stock1    Stock2    Stock3    Stock4    Stock5
0 2024-01-01  0.391449 -0.097651  0.388190  0.019484  0.298529
1 2024-01-02  0.064482  1.565104 -0.925339  2.035023 -1.739270
2 2024-01-03 -1.439431  1.297013  0.181790  1.186281 -0.225653
3 2024-01-04 -0.547698  0.523858  0.702089  0.602619 -0.280867
4 2024-01-05  0.337418  0.183443  0.427368 -0.328869  0.380640
